In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn.metrics import log_loss
import xgboost as xgb
import lightgbm as lgb
import gc
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'xgboost'

# **1. Data Preparation**

In [ ]:
print('loading files...')
train = pd.read_csv('./DATA/DATA_V6/stock_train_data_20171006.csv')
test = pd.read_csv('./DATA/DATA_V6/stock_test_data_20171006.csv')

train = train.drop(['weight','group','era'], axis=1)  
test = test.drop('group', axis=1)
print ("Dimension of train {}".format(train.shape))
print ("Dimension of test {}".format(test.shape))

# 2. Training/Predicting Pipeline

In [ ]:
# xgb
params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 
          'objective': 'binary:logistic', 'silent': True}

X = train.drop(['id','label'], axis=1)
features = X.columns
X = X.values
y = train['label'].values
sub=test['id'].to_frame()
sub['proba']=0

In [ ]:
nrounds=200  # need to change to 2000
kfold = 2  # need to change to 5
skf = StratifiedKFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    d_train = xgb.DMatrix(X_train, y_train) 
    d_valid = xgb.DMatrix(X_valid, y_valid) 
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    xgb_model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=100, 
                           maximize=True, verbose_eval=100)
    sub['proba'] += xgb_model.predict(xgb.DMatrix(test[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (2*kfold)
gc.collect()
sub.head(2)

In [ ]:
# lgb
params = {'metric': 'auc', 'learning_rate' : 0.01, 'max_depth':10, 'max_bin':10,  'objective': 'binary', 
          'feature_fraction': 0.8,'bagging_fraction':0.9,'bagging_freq':10,  'min_data': 500}

skf = StratifiedKFold(n_splits=kfold, random_state=1)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' lgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    lgb_model = lgb.train(params, lgb.Dataset(X_train, label=y_train), nrounds, 
                  lgb.Dataset(X_eval, label=y_eval), verbose_eval=100, 
                  early_stopping_rounds=100)
    sub['proba'] += lgb_model.predict(test[features].values, 
                        num_iteration=lgb_model.best_iteration) / (2*kfold)

# 3. Put submission to csv file

In [ ]:
from datetime import datetime
sub.to_csv('./submission/sub{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')))